# Aggregate
## Weather Observation Station 20

```sql
SELECT ROUND(AVG(T.LAT_N),4) FROM
(SELECT LAT_N, ROW_NUMBER() OVER(ORDER BY LAT_N) AS ROW_NUM FROM STATION ORDER BY LAT_N) AS T
WHERE T.ROW_NUM = ANY (SELECT FLOOR(COUNT(*)/2+0.5) FROM STATION) OR
T.ROW_NUM = ANY (SELECT CEIL(COUNT(*)/2+0.5) FROM STATION);
```

# Basic Joins
## The Report
```sql
SELECT S.Name, G.Grade, S.Marks
FROM Students as S
INNER JOIN Grades as G ON S.Marks BETWEEN G.Min_Mark AND G.Max_Mark
WHERE G.Grade >= 8
ORDER BY G.Grade DESC, S.Name;
```

## Top Competitors
```sql
SELECT H.hacker_id, H.name
FROM Submissions AS S
INNER JOIN Hackers AS H ON S.hacker_id = H.hacker_id
INNER JOIN Challenges AS C ON C.challenge_id = S.challenge_id
INNER JOIN Difficulty AS D ON C.difficulty_level = D.difficulty_level
WHERE S.score = D.score
GROUP BY H.hacker_id, H.name
HAVING COUNT(H.hacker_id) > 1
ORDER BY COUNT(H.hacker_id) DESC, H.hacker_id ASC;
```

## Ollivander's Inventory
```sql
SELECT Wands1.id, Prop1.age, Wands1.coins_needed, Wands1.power FROM Wands AS Wands1
JOIN Wands_Property AS Prop1 ON Wands1.code = Prop1.code
WHERE Prop1.is_evil = 0
AND Wands1.coins_needed = (SELECT MIN(Wands2.coins_needed) FROM Wands AS Wands2 JOIN Wands_Property AS Prop2 ON Wands2.code = Prop2.code WHERE
                             Wands1.power = Wands2.power AND Prop1.age = Prop2.age)
ORDER BY Wands1.power DESC, Prop1.age DESC;
```

I don't think hackerrank supports OVER
```sql
SELECT MIN(Wands.coins_needed) OVER(PARTITION BY Wands.power, Prop.age) AS mincoins
FROM Wands
INNER JOIN Wands_Property AS Prop ON Wands.code = Prop.code;
```

## Challenges
```sql
SELECT h.hacker_id, h.name, COUNT(c.challenge_id) as cnt
FROM Hackers AS h JOIN Challenges AS c ON h.hacker_id = c.hacker_id
GROUP BY h.hacker_id, h.name
HAVING
cnt = (SELECT MAX(temp1.cnt) FROM
(SELECT COUNT(*) as cnt FROM Challenges
GROUP BY hacker_id) AS temp1)
OR
cnt IN (SELECT temp2.cnt FROM
(SELECT hacker_id, COUNT(*) as cnt FROM Challenges
GROUP BY hacker_id) AS temp2
GROUP BY temp2.cnt
HAVING COUNT(temp2.cnt) = 1)
ORDER BY COUNT(c.challenge_id) DESC, h.hacker_id;
```

Things I learned here:
* You have to think recursively when constructing SQL queries - multiple layers of processing means recursive `SELECT FROM table1` statements, where `table1` itself is an alias for a `SELECT FROM table2` statement.
* When constructing recursive queries, you need to use aliasing to access the columns of the returned subquery.

## Contest Leaderboards (Basic Join)
```sql
SELECT hack.hacker_id, hack.name, SUM(sub.max_score) as total_score
FROM Hackers as hack JOIN (SELECT Submissions.hacker_id, Submissions.challenge_id, MAX(Submissions.score) as max_score
FROM Submissions
GROUP BY Submissions.hacker_id, Submissions.challenge_id) AS sub ON hack.hacker_id = sub.hacker_id
GROUP BY hack.hacker_id, hack.name
HAVING total_score > 0
ORDER BY total_score DESC, hack.hacker_id;
```

# Advanced Join
## SQL Project Planning
```sql
/* Solution without JOIN or recursion
The two subqueries return single-column tables. The top-level query returns all possible 2-value combinations
combined from the values of each single-column table. For example, if both queries return 11-record tables,
then the top-level query returns 121 records. */
SELECT S.Start_Date AS Proj_Start_Date, MIN(E.End_Date) AS Proj_End_Date FROM
(SELECT Start_Date FROM Projects WHERE Start_Date NOT IN (SELECT End_Date FROM Projects)) AS S,
(SELECT End_Date FROM Projects WHERE End_Date NOT IN (SELECT Start_Date FROM Projects)) AS E
WHERE S.Start_Date < E.End_Date
GROUP BY S.Start_Date
ORDER BY Proj_End_Date-Proj_Start_Date, Proj_Start_Date;
```
```sql
/* Requires recursive queries via CTE */

/* Sanity check */
-- SELECT P1.Start_Date, P1.End_Date, P2.Start_Date, P1.End_Date-P1.Start_Date FROM Projects as P1
-- JOIN Projects as P2 ON P1.End_Date = P2.Start_Date;

-- WITH TaskTable AS (SELECT * FROM Projects WHERE Start_Date='2015-10-01' LIMIT 1) UNION ALL
-- (SELECT * FROM Projects WHERE TaskTable.End_Date = Projects.Start_Date);

/* Recursive CTE from MySQL dev guide */
-- WITH RECURSIVE cte (n) AS
-- (
--   SELECT 1
--   UNION ALL
--   SELECT n + 1 FROM cte WHERE n < 5
-- )
-- SELECT * FROM cte;

-- SELECT * FROM Projects WHERE Start_Date='2015-10-01' LIMIT 1;
-- WITH RECURSIVE cte AS
-- (
--   SELECT 1 AS n, 'abc' AS str
--   UNION ALL
--   SELECT n + 1, CONCAT(str, str) FROM cte WHERE n < 3
-- )
-- SELECT * FROM cte;

-- WITH RECURSIVE TaskTable (Task_ID, Start_Date, End_Date) AS 
-- ((SELECT * FROM Projects WHERE Start_Date='2015-10-01' LIMIT 1)
--  UNION ALL
--  (SELECT * FROM Projects WHERE TaskTable.End_Date = Projects.Start_Date)
-- )
-- SELECT * FROM TaskTable;

/* Recursive query for a single project */
-- SELECT * FROM (
-- WITH RECURSIVE TaskTable (Task_ID, Start_Date, End_Date) AS 
-- ((SELECT * FROM Projects WHERE Start_Date='2015-10-01' LIMIT 1)
--  UNION ALL
--  (SELECT P.Task_ID, P.Start_Date, P.End_Date FROM Projects AS P JOIN TaskTable ON TaskTable.End_Date = P.Start_Date)
-- )
-- SELECT MIN(TaskTable.Start_Date), MAX(TaskTable.End_Date) FROM TaskTable
-- ) AS OneProject;

/* We will traverse from earliest to latest project */
-- SELECT * FROM Projects ORDER BY Start_Date;

SELECT Project_Start_Date, Project_End_Date FROM (
WITH RECURSIVE TaskTable (Task_ID, Start_Date, End_Date) AS 
((SELECT * FROM Projects WHERE Start_Date='2015-10-01' LIMIT 1)
 UNION ALL
 (SELECT P.Task_ID, P.Start_Date, P.End_Date FROM Projects AS P JOIN TaskTable ON TaskTable.End_Date = P.Start_Date)
)
SELECT MIN(TaskTable.Start_Date) AS Project_Start_Date, MAX(TaskTable.End_Date) AS Project_End_Date FROM TaskTable
) AS OneProject;


WITH RECURSIVE ProjectTable (Project_Start_Date, Project_End_Date) AS (
( /* Get all tasks for first project */
SELECT Project_Start_Date, Project_End_Date FROM (
WITH RECURSIVE TaskTable (Task_ID, Start_Date, End_Date) AS 
((SELECT * FROM Projects ORDER BY Start_Date LIMIT 1)
 UNION ALL
 (SELECT P.Task_ID, P.Start_Date, P.End_Date FROM Projects AS P JOIN TaskTable ON TaskTable.End_Date = P.Start_Date)
)
SELECT MIN(TaskTable.Start_Date) AS Project_Start_Date, MAX(TaskTable.End_Date) AS Project_End_Date FROM TaskTable
) AS OneProject)
UNION ALL
( /* Get all tasks for subsequent projects */
SELECT Project_Start_Date, Project_End_Date FROM (
WITH RECURSIVE TaskTable (Task_ID, Start_Date, End_Date) AS 
((SELECT * FROM Projects WHERE Start_Date > (SELECT MAX(Project_End_Date) FROM ProjectTable) ORDER BY Start_Date LIMIT 1)
 UNION ALL
 (SELECT P.Task_ID, P.Start_Date, P.End_Date FROM Projects AS P JOIN TaskTable ON TaskTable.End_Date = P.Start_Date)
)
SELECT MIN(TaskTable.Start_Date) AS Project_Start_Date, MAX(TaskTable.End_Date) AS Project_End_Date FROM TaskTable
) AS OneProject)
)
SELECT * FROM ProjectTable;
```

## Placements
```sql
SELECT Stu.Name
FROM ((Students Stu JOIN Friends Fr ON Stu.ID = Fr.ID)
JOIN Packages Pkg1 ON Pkg1.ID = Stu.ID)
JOIN Packages Pkg2 ON Pkg2.ID = Fr.Friend_ID
WHERE Pkg1.Salary < Pkg2.Salary
ORDER BY Pkg2.Salary;
```

## Symmetric Pairs
```sql
SELECT F1.X, F1.Y FROM Functions F1 JOIN Functions F2 ON F1.X = F2.Y AND F1.Y = F2.X
GROUP BY F1.X, F1.Y
HAVING COUNT(F1.X) > 1 OR F1.X < F1.Y
ORDER BY F1.X;
```

# Advanced Select
## Occupations
* `:=` is the assignment operator (`=` is the comparison operator). Use this outside of `SET` statements to assign values. `=` can only be used for assignment within a `SET` statement. https://dev.mysql.com/doc/refman/8.0/en/assignment-operators.html#operator_assign-value
* Apparently you can `ORDER BY Name` from OCCUPATIONS even though the new table does not contain a Name column.



```sql
-- Split names into columns based on occupation
SELECT 
CASE WHEN Occupation='Doctor' THEN Name END AS Doctor,
CASE WHEN Occupation='Professor' THEN Name END AS Professor,
CASE WHEN Occupation='Singer' THEN Name END AS Singer,
CASE WHEN Occupation='Actor' THEN Name END AS Actor
FROM OCCUPATIONS
ORDER BY Name;

-- For each new record, calculate the index
-- E.g. rs=3 means the third singer
SET @rd:=0, @rp:=0, @rs:=0, @ra:=0;

SELECT CASE 
WHEN Occupation='Doctor' THEN @rd:=@rd+1
WHEN Occupation='Professor' THEN @rp:=@rp+1
WHEN Occupation='Singer' THEN @rs:=@rs+1
WHEN Occupation='Actor' THEN @ra:=@ra+1
END AS rn
FROM OCCUPATIONS;

-- Final
SET @rd:=0, @rp:=0, @rs:=0, @ra:=0;

SELECT MIN(Doctor), MIN(Professor), MIN(Singer), MIN(Actor) FROM
(SELECT
CASE 
WHEN Occupation='Doctor' THEN @rd:=@rd+1
WHEN Occupation='Professor' THEN @rp:=@rp+1
WHEN Occupation='Singer' THEN @rs:=@rs+1
WHEN Occupation='Actor' THEN @ra:=@ra+1
END AS rn,
CASE WHEN Occupation='Doctor' THEN Name END AS Doctor,
CASE WHEN Occupation='Professor' THEN Name END AS Professor,
CASE WHEN Occupation='Singer' THEN Name END AS Singer,
CASE WHEN Occupation='Actor' THEN Name END AS Actor
FROM OCCUPATIONS
ORDER BY Name) temp
GROUP BY rn;
```

# Alternative Queries
## Draw the Triangle 1
```sql
WITH RECURSIVE cte (n) AS (
    SELECT 20
    UNION ALL
    SELECT n-1 FROM cte WHERE n>0
)
SELECT REPEAT("* ", n) FROM cte;
```